In [1]:
import os
import json
import dotenv
from crewai import Crew, Agent, Task, tools
from langchain_community.chat_models import ChatLiteLLM
import litellm
# import XDATA
from XDATA import create_x_data_collector_task

dotenv.load_dotenv()

True

In [2]:



class XSentimentAnalysisProject:
    def __init__(self, model_provider='openai'):
        """
        Initialize the X Sentiment Analysis Project with configurable LLM.
        
        Args:
            model_provider (str): LLM provider (openai, anthropic, etc.)
        """
        # Configure LLM with litellm for flexibility
        self.llm = ChatLiteLLM(model="gpt-3.5-turbo")

    def create_x_data_collector_agent(self):
        """
        Create an agent responsible for collecting X (Twitter) data.
    
        Returns:
            Agent: X Data Collector Agent
        """
        return Agent(
            role='X Data Collector',
            goal='Retrieve comprehensive tweet data from target X creators',
            backstory="""You are a sophisticated data gathering expert specialized 
            in social media content extraction. Your mission is to collect tweets, 
            analyze their context, and prepare structured data for further analysis.""",
            tools=[
                # Replace web scraping tools with our custom XDataScraper
                    tools.PythonTool(
                    description="Use XDataScraper to collect tweets from specified X creators",
                    # Point to the create_x_data_collector_task function from the scraper
                    python_function="create_x_data_collector_task"
                )
            ],
            verbose=True,
            allow_delegation=True
        )

    def create_sentiment_analyzer_agent(self):
        """
        Create an agent for sentiment and contextual analysis of collected tweets.
        
        Returns:
            Agent: Sentiment Analyzer Agent
        """
        return Agent(
            role='Social Media Sentiment Analyst',
            goal='Perform deep sentiment and thematic analysis on collected tweet data',
            backstory="""You are an expert in natural language processing and 
            social media content interpretation. Your advanced analytical skills 
            help extract nuanced insights from digital communication.""",
            tools=[
                tools.RAGTool(),  # CrewAI's RAG tool for contextual analysis
            ],
            verbose=True,
            allow_delegation=True
        )

    def create_data_collection_task(self, data_collector_agent, creators):
        """
        Design a task for collecting tweet data from specified creators.
    
        Args:
            data_collector_agent (Agent): X Data Collector Agent
            creators (list): List of X creator usernames
    
        Returns:
            Task: Data collection task
        """
        return Task(
            description=f"""Collect tweets from the following X creators: {', '.join(creators)}
            - Use XDataScraper to scrape tweets
            - Collect comprehensive tweet metadata including:
              * Tweet content
              * Timestamp
              * Engagement metrics (likes, retweets, quotes)
              * User information
            - Ensure data is stored in a structured JSON format
            - Respect data privacy and platform terms of service""",
            agent=data_collector_agent,
            tools=[
                tools.PythonTool(
                    description="Execute tweet data collection",
                    python_function="create_x_data_collector_task"
                )
            ],
            expected_output="""Comprehensive JSON structure containing:
            {
                "data_source": "X (Twitter)",
                "users_analyzed": ["username1", "username2"],
                "total_tweets_collected": 100,
                "tweets": [
                    {
                        "username": "creator_username",
                        "content": "Tweet text content",
                        "created_at": "ISO 8601 timestamp",
                        "likes": 42,
                        "retweets": 10,
                        "quote_count": 5,
                        "reply_count": 3,
                        "language": "en"
                    }
                ]
            }"""
        )

    def create_sentiment_analysis_task(self, sentiment_analyzer_agent, collected_data):
        """
        Design a task for sentiment and thematic analysis.
        
        Args:
            sentiment_analyzer_agent (Agent): Sentiment Analyzer Agent
            collected_data (dict): JSON data of collected tweets
        
        Returns:
            Task: Sentiment analysis task
        """
        return Task(
            description="""Perform comprehensive sentiment and thematic analysis on collected tweets
            - Analyze sentiment for each tweet (positive, negative, neutral)
            - Identify dominant themes and topics across tweets
            - Generate a detailed PDF report with insights
            - Include visualizations of sentiment distribution
            - Provide actionable insights about each creator's content""",
            agent=sentiment_analyzer_agent,
            context=[collected_data],
            expected_output="""PDF Report with:
            1. Overall sentiment analysis
            2. Creator-specific insights
            3. Thematic breakdown
            4. Engagement metrics visualization"""
        )

    def run_analysis(self, creators):
        """
        Execute the complete X sentiment analysis workflow.
        
        Args:
            creators (list): List of X creator usernames
        """
        collected_data = create_x_data_collector_task(creators)

        # Initialize agents
        data_collector = self.create_x_data_collector_agent()
        sentiment_analyzer = self.create_sentiment_analyzer_agent()

        # Create sentiment analysis task with the collected data
        sentiment_analysis_task = self.create_sentiment_analysis_task(
            sentiment_analyzer, 
            collected_data  # Pass the actual collected data instead of an empty placeholder
        )

        # Create crew and execute workflow
        crew = Crew(
            agents=[sentiment_analyzer],  # Remove data collector since scraping is now pre-done
            tasks=[sentiment_analysis_task],
            verbose=2
        )

        # Run the workflow
        result = crew.kickoff()
        return result

def main():
    # Example X creators to analyze
    x_creators = [
        'elonmusk', 'naval', 'paulg', 
        'sama', 'lex_fridman', 'balajis'
    ]

    # Initialize and run project
    project = XSentimentAnalysisProject()
    analysis_results = project.run_analysis(x_creators)
    
    # Optional: Save or further process results
    print("Analysis Complete:", analysis_results)



In [ ]:
if __name__ == "__main__":
    main()

AttributeError: module 'crewai.tools' has no attribute 'PythonTool'

2025-01-23 22:26:35.531 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:301 - No account available for queue "SearchTimeline". Next available at 22:41:35
2025-01-23 22:26:35.542 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:301 - No account available for queue "SearchTimeline". Next available at 22:41:35
2025-01-23 22:26:35.548 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:301 - No account available for queue "SearchTimeline". Next available at 22:41:35
2025-01-23 22:26:35.550 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:301 - No account available for queue "SearchTimeline". Next available at 22:41:35
2025-01-23 22:26:35.556 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:301 - No account available for queue "SearchTimeline". Next available at 22:41:35


Error scraping tweets for elonmusk: 'Tweet' object has no attribute 'createdAt'


2025-01-23 22:26:40.550 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:308 - Continuing with account hellmate9080 on queue SearchTimeline


Error scraping tweets for naval: 'Tweet' object has no attribute 'createdAt'


...

Error scraping tweets for sama: 'Tweet' object has no attribute 'createdAt'


2025-01-23 22:26:56.668 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:308 - Continuing with account hellmate9080 on queue SearchTimeline


Error scraping tweets for paulg: 'Tweet' object has no attribute 'createdAt'


2025-01-23 22:27:02.059 | INFO     | twscrape.accounts_pool:get_for_queue_or_wait:308 - Continuing with account hellmate9080 on queue SearchTimeline


Error scraping tweets for balajis: 'Tweet' object has no attribute 'createdAt'
